In [3]:
# import libraries
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import math
from sklearn.metrics import mean_squared_error
from numpy import array
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import pearsonr, kurtosis, skew
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from xgboost import XGBClassifier

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [ ]:
# buat fungsi untuk merubah timeseries data menjadi input X dan output y, dengan teknik sliding window
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix], sequences[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
# fungsi tambahan untuk membuat statistical features, semoga meningkatkan akurasi
def stats_features(input_data):
    inp = list()
    for i in range(len(input_data)):
        inp2 = list()
        inp2 = input_data[i]
        min=float(np.min(inp2))
        max=float(np.max(inp2))
        diff=float(max-min)
        std=float(np.std(inp2))
        mean=float(np.mean(inp2))
        median=float(np.median(inp2))
        kurt=float(kurtosis(inp2))
        sk=float(skew(inp2))
        inp2=np.apped(inp2,min)
        inp2=np.apped(inp2,max)
        inp2=np.apped(inp2,diff)
        inp2=np.apped(inp2,std)
        inp2=np.apped(inp2,mean)
        inp2=np.apped(inp2,median)
        inp2=np.apped(inp2,kurt)
        inp2=np.apped(inp2,sk)
        # print(list(inp2))
        inp=np.append(inp,inp2)
    inp=inp.reshape(len(input_data),-1)
    # print(inp)
    return inp

In [ ]:
# model untuk neural network (MLP multi layer perceptron)
def mlp(X_train, X_test, y_train, y_test):
    mlp_model = MLPRegressor(random_state=42)
    mlp_model.fit(X_train, y_train)
    # get the predictions
    y_pred = mlp_model.predict(X_test)
    y_pred = np.round(y_pred, 0)
    # get rmse for the model
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    # get pearson correlation
    corr, p_value = pearsonr(y_test, y_pred)
    # returning the rmse, correlation, and the predictions result
    return rmse, corr, y_pred


In [ ]:
# model untuk KNN (K Nearest Neighbors)
def knn(X_train, X_test, y_train, y_test):
    knn_model = KNeighborsRegressor(n_neighbors=3)
    knn_model.fit(X_train, y_train)
    # get the predictions
    y_pred = knn_model.predict(X_test)
    y_pred = np.round(y_pred, 0)
    # get rmse for the model
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    # get pearson correlation
    corr, p_value = pearsonr(y_test, y_pred)
    # returning the rmse, correlation, and the predictions result
    return rmse, corr, y_pred

In [ ]:
# model untuk decision tree
def dt(X_train, X_test, y_train, y_test):
    dt_model = DecisionTreeRegressor(random_state=42)
    dt_model.fit(X_train, y_train)
    # get the predictions
    y_pred = dt_model.predict(X_test)
    y_pred = np.round(y_pred, 0)
    # get rmse for the model
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    # get pearson correlation
    corr, p_value = pearsonr(y_test, y_pred)
    # returning the rmse, correlation, and the predictions result
    return rmse, corr, y_pred

In [ ]:
# model untuk SVR (Support Vector Regression)
def svr(X_train, X_test, y_train, y_test):
    svr_model = SVR()
    svr_model.fit(X_train, y_train)
    # get the predictions
    y_pred = svr_model.predict(X_test)
    y_pred = np.round(y_pred, 0)
    # get rmse for the model
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    # get pearson correlation
    corr, p_value = pearsonr(y_test, y_pred)
    # returning the rmse, correlation, and the predictions result
    return rmse, corr, y_pred

In [ ]:
# model untuk random forest
def rf(X_train, X_test, y_train, y_test):
    rf_model = RandomForestRegressor(random_state=42)
    rf_model.fit(X_train, y_train)
    # get the predictions
    y_pred = rf_model.predict(X_test)
    y_pred = np.round(y_pred, 0)
    # get rmse for the model
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    # get pearson correlation
    corr, p_value = pearsonr(y_test, y_pred)
    # returning the rmse, correlation, and the predictions result
    return rmse, corr, y_pred

In [ ]:
# import dataset
data = pd.read_csv('bikesharing_day.csv')